In [8]:
import findspark
findspark.init()
findspark.find()

'C:\\spark-3.2.1-bin-hadoop3.2'

In [9]:
import pyspark
from pyspark import SparkContext
from pyspark import SparkConf

In [10]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [11]:
df = spark.read.format("json").option("multiLine", "true").load("reviews.json")

# Avoiding Shuffle and Reducing Operational Expenses

In [12]:
review = df.select('paper.review').collect()[0][0]

In [14]:
confidence_col = []
evaluation_col = []
orientation_col = []
timespan_col= []
for i in review:
    for j in i:
        confidence_col.append(j[0])
        evaluation_col.append(j[1])
        orientation_col.append(j[4])
        timespan_col.append(j[7])

In [15]:
import pandas as pd
from pyspark.sql import SQLContext

# intialise data of lists.
data = {'confidence':confidence_col,
       'evaluation':evaluation_col,'orientation':orientation_col,'timespan':timespan_col}
  
# Create DataFrame
review_data = pd.DataFrame(data)
review_data = review_data.dropna()
review_data ['confidence']= review_data[['confidence']].astype(int)
review_data ['orientation']= review_data[['orientation']].astype(int)
review_data ['evaluation']= review_data[['evaluation']].astype(int)

review_data = spark.createDataFrame(data=review_data)
review_data.show()


+----------+----------+-----------+----------+
|confidence|evaluation|orientation|  timespan|
+----------+----------+-----------+----------+
|         4|         1|          0|2010-07-05|
|         4|         1|          1|2010-07-05|
|         5|         1|          1|2010-07-05|
|         4|         2|          1|2010-07-05|
|         4|         2|          0|2010-07-05|
|         4|         2|          0|2010-07-05|
|         4|         2|          1|2010-07-05|
|         3|         2|          1|2010-07-05|
|         3|         0|         -1|2010-07-05|
|         4|         2|          2|2010-07-05|
|         2|        -2|         -1|2010-07-05|
|         4|         2|          1|2010-07-05|
|         4|         2|          0|2010-07-05|
|         5|         2|          1|2010-07-05|
|         4|        -1|          0|2010-07-05|
|         4|        -2|         -1|2010-07-05|
|         4|         1|         -1|2010-07-05|
|         5|        -2|         -1|2010-07-05|
|         4| 

In [16]:
review_rdd=review_data.rdd
# //at this point data is spread randomly, records for the same user_id can be on different executors
# //when
# //shuffle -> all records for the same user_id go to the same executor

review_rdd = review_rdd.keyBy(lambda x: x[0])

# //other operations that are key-wise doesn't require shuffle
review_rdd.collect()

[(4, Row(confidence=4, evaluation=1, orientation=0, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=1, orientation=1, timespan='2010-07-05')),
 (5, Row(confidence=5, evaluation=1, orientation=1, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=1, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=0, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=0, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=1, timespan='2010-07-05')),
 (3, Row(confidence=3, evaluation=2, orientation=1, timespan='2010-07-05')),
 (3, Row(confidence=3, evaluation=0, orientation=-1, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=2, timespan='2010-07-05')),
 (2, Row(confidence=2, evaluation=-2, orientation=-1, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=1, timespan='2010-07-05')),
 (4, Row(confidence=4, evaluation=2, orientation=0, timespan='2010-07-05'